In [6]:
import tkinter as tk
from PIL import Image, ImageTk
import os
import threading 
import spacy
from spacy import displacy
import matplotlib.pyplot as plt
import networkx as nx
import snakes.plugins
snakes.plugins.load("gv", "snakes.nets", "nets")
from nets import *
import re
from collections import defaultdict
import time

nlp = spacy.load("en_core_web_sm")


In [7]:
together_words = {"and", "both", "along with", "as well as", "in conjunction with", "together with"}
individual_words = {"or", "either", "one of", "neither", "nor", "any of", "each of"}

In [8]:
#Entities

class Preposition:
    def __init__(self, name, tag=None):
        self.name = name
        self.tag = tag  # Stores the syntactic role (e.g., "prep", "mark")

    def __str__(self):
        return f"({self.name}, TAG: {self.tag})" if self.tag else f"({self.name})"

    
class Verb:
    def __init__(self, name, preposition=None, tag=None):
        self.name = name
        self.preposition = [preposition] if preposition and not isinstance(preposition, list) else (preposition or [])
        self.tag = tag  # Stores the syntactic role of the verb (e.g., "advcl", "root")

    def __str__(self):
        prepositions = ', '.join(str(p) for p in self.preposition) if self.preposition else ''
        tag_str = f"TAG: {self.tag}" if self.tag else ''
        
        components = [self.name]
        if prepositions:
            components.append(f"Prepositions: {prepositions}")
        if tag_str:
            components.append(tag_str)
        
        return ', '.join(components)


class Noun:
    def __init__(self, name, preposition=None):
        self.name = name
        self.preposition = preposition if isinstance(preposition, list) else [preposition]
    
    def __str__(self):
        if self.preposition:
            p = ', '.join(str(p) for p in self.preposition)
            return f"{self.name}, {p}"
        else:
            return f"{self.name}"

    
class Task:
    def __init__(self, verbs, nouns, subtask=None):
        self.verbs = verbs if isinstance(verbs, list) else [verbs] 
        self.nouns = nouns if isinstance(nouns, list) else [nouns]
        self.subtask = subtask if isinstance(subtask, Task) else None  # Optional subtask

    def __str__(self):
        verb_str = ', '.join(str(verb) for verb in self.verbs)
        nouns_str = ', '.join(str(noun) for noun in self.nouns)
        subtask_str = f", Subtask: ({self.subtask})" if self.subtask else ""

        return f"Task(Verbs: [{verb_str}], Nouns: [{nouns_str}]{subtask_str})"

    def get_all_actors(self):
        actors = [noun.name for noun in self.nouns]
        if self.subtask:
            actors.extend(self.subtask.get_all_actors())  # Include subtask actors
        return actors



preposition1 = Preposition("on")
noun1 = Noun("table", preposition1)
noun2 = Noun("chair")

# Create a list of verbs
verbs = Task(["places", "moves"], [noun1, noun2])

# Print out the verb and its related nouns and prepositions
print(verbs)  # Output: Verb(places, moves, Noun(table, floor, Preposition(on)), Noun(chair))


Task(Verbs: [places, moves], Nouns: [table, (on), chair, None])


In [9]:
def analyse_tree(doc):

    all_tasks = []

    def add_actors(subj):
        prepositions = []
        collect_prepositions(subj, prepositions)
        actor = Noun(subj.text, prepositions)
        return actor
    
    def add_task(verbs, actors):
        task = Task(verbs, actors)
        return task
    
    def add_verb(verb, tag):
        prepositions = []
        collect_prepositions(verb, prepositions)  
        verb1 = Verb(verb.text, preposition=prepositions, tag=tag)
        return verb1

    def collect_subj_conjuncts(subj, actors):
        for child in subj.children:
            if child.dep_ == "conj":
                actor = add_actors(child)
                actors.append(actor)
                collect_subj_conjuncts(child, actors)
    
    def collect_prepositions(subj, prepositions):
        for child in subj.children:
            if child.dep_ == "cc":
                preposition = Preposition(child.text, tag="cc")
                prepositions.append(preposition)

            if child.dep_ == "mark":
                preposition = Preposition(child.text, tag="mark")
                prepositions.append(preposition)
            
            if child.dep_ == "advmod":
                preposition = Preposition(child.text, tag="advmod")
                prepositions.append(preposition)
            

    def collect_conjuncts(verb, task):
        for child in verb.children:
            if child.dep_ == "conj" and child.pos_ == "VERB":
                conj_nsubj = []

                # Check if the conjunct verb has its own subject
                for grandchild in child.children:
                    if grandchild.dep_ == "nsubj":
                        actor = add_actors(grandchild)
                        conj_nsubj.append(actor)
                        collect_subj_conjuncts(grandchild, conj_nsubj)

                 # Add a tag 
                verb = add_verb(child, tag="conj")

                # If no subject for the conjunct verb, add it to the root's verbs
                if not conj_nsubj:
                    task.verbs.append(verb) 
                else:
                    task = add_task(verb, conj_nsubj)
                    all_tasks.append(task)

                collect_conjuncts(child, task)

            if child.dep_ == "advcl" and child.pos_ == "VERB":
                conj_nsubj = []
                # Check if the sub verb has its own subject
                for grandchild in child.children:                   
                    if grandchild.dep_ == "nsubj":
                        actor = add_actors(grandchild)
                        conj_nsubj.append(actor)
                        collect_subj_conjuncts(grandchild, conj_nsubj)
                # Add a tag 
                verb = add_verb(child, tag="advcl")

                # Add a new subtask
                if not conj_nsubj:         
                    subtask = add_task(verb, task.nouns)
                else:
                    subtask = add_task(verb, conj_nsubj)
                task.subtask = subtask


    for token in doc:
        if token.dep_ == "ROOT":
            verb = add_verb(token, tag="ROOT")
            verb_data = [verb]  # Start with the root verb
            actors_list = []

            for child in token.children:
                if child.dep_ == "nsubj":
                    actor = add_actors(child)
                    actors_list.append(actor)
                    collect_subj_conjuncts(child, actors_list)

            task = add_task(verb_data, actors_list)
            all_tasks.append(task)
            collect_conjuncts(token, task)

    return all_tasks


In [10]:
input_text = "Bob waters the plants, while Anna cleans the room."


doc = nlp(input_text)
all_tasks = analyse_tree(doc)
for i in all_tasks:
    print(i)



Task(Verbs: [waters, TAG: ROOT], Nouns: [Bob], Subtask: (Task(Verbs: [cleans, Prepositions: (while, TAG: mark), TAG: advcl], Nouns: [Anna])))


In [11]:
parallel_markers = ["while", "as", "when", "whilst"]
after_completing_markers = ["after", "once"]
before_completing_markers = ["before", "until"]


In [12]:
def is_or_exist(words):
    for word in words:
        for p in word.preposition:
            if p.name == "or":
                return True
    return False

def find_subtask(task):
    if task.subtask is None:  
        return None  

    for verb in task.subtask.verbs:
        if verb.preposition:  
            return verb.preposition[0]  

    return None 

def is_this_or(actor):
    for p in actor.preposition:
        if p.name == "or":
            return True
    return False

def is_and_exist(actors):
    for actor in actors:
        for p in actor.preposition:
            if p.name == "and":
                return True
    return False

def print_names(actors):
    s = ""
    for a in actors:
        s+= a.name+("_")
    return s

def is_and_exist(actors):
    for actor in actors:
        for p in actor.preposition:
            if p.name == "and":
                return True
    return False

def initialize_actors(all_tasks):
    actors_actions = {}
    for task in all_tasks:
        for actor in task.nouns:
            if actor.name not in actors_actions:
                actors_actions[actor.name] = []
        for actors in task.subtask.nouns:
            if actor.name not in actors_actions:
                actors_actions[actor.name] = []


#rules for created dictionary
def get_parallel_tasks(doc):
    order_cnt = 0
    actors_actions = defaultdict(list)
    all_tasks = analyse_tree(doc)
    print(all_tasks)


    for task in all_tasks:
        
        #if task has subtask 
        subtask_prep = find_subtask(task)    
        if (subtask_prep):
            subtask = task.subtask

            #parallel synchronized execution
            if subtask_prep.name in parallel_markers:
                actors = list(set(task.nouns + subtask.nouns))
                names_str = print_names(actors)

                for actor in task.nouns:                                               
                    actors_actions[actor.name].append("BARRIER_" + names_str + str(order_cnt))
                    for verb in task.verbs:
                        actors_actions[actor.name].append(verb.name)
                    actors_actions[actor.name].append("SINC_"+ names_str + str(order_cnt))
                for actor in subtask.nouns:                                               
                    actors_actions[actor.name].append("BARRIER_" + names_str + str(order_cnt))
                    for verb in subtask.verbs:
                        actors_actions[actor.name].append(verb.name)
                    actors_actions[actor.name].append("SINC_"+ names_str + str(order_cnt))    
                order_cnt+=1
                continue
            
            #direct order
            #one action directly after another
            if subtask_prep.name in after_completing_markers:
                actors = list(set(task.nouns + subtask.nouns))
                names_str = print_names(actors)
                for actor in task.nouns:                                               
                    actors_actions[actor.name].append("BARRIER_" + names_str + str(order_cnt))
                    for verb in task.verbs:
                        actors_actions[actor.name].append(verb.name)
                for actor in subtask.nouns:                                               
                    actors_actions[actor.name].append(subtask.verbs[0].name)
                    actors_actions[actor.name].append("BARRIER_" + names_str + str(order_cnt))
                order_cnt+=1
                continue
            
            #direct order
            #one action directly before another
            if subtask_prep.name == "before":
                actors = list(set(task.nouns + subtask.nouns))
                names_str = print_names(actors)
                for actor in task.nouns: 
                    for verb in task.verbs:
                        actors_actions[actor.name].append(verb.name)
                    actors_actions[actor.name].append("BARRIER_" + names_str + str(order_cnt))                          

                for actor in subtask.nouns:                                               
                    actors_actions[actor.name].append("BARRIER_" + names_str + str(order_cnt))
                    actors_actions[actor.name].append(subtask.verbs[0].name)
                order_cnt+=1
                continue

        if len(task.nouns) > 1:
            #has OR between verbs     
            if (is_or_exist(task.verbs)):
                verbs_str = print_names(task.verbs)
                for actor in task.nouns:                             
                    if actor.name not in actors_actions:
                        actors_actions[actor.name].append("ORVERB_"+ verbs_str  + str(order_cnt))                                  
            else:        
                for verb in task.verbs:
                    #task choice
                    if (is_or_exist(task.nouns)): #gather actors groups if there is OR 
                        actors_before = []
                        groups = ""
                        for actor in task.nouns:
                            if actor.name not in actors_actions:
                                actors_before.append(actor)
                            if (is_this_or(actor)):
                                if (is_and_exist(actors_before)):
                                    names_str = print_names(actors_before)
                                    for a in actors_before:
                                        actors_actions[a.name].append("ORNOUN_" + names_str + str(order_cnt))
                                        actors_actions[a.name].append(verb.name)
                                else:
                                    for a in actors_before:
                                        actors_actions[a.name].append("ORNOUN_" + a.name + "_" + str(order_cnt))
                                        actors_actions[a.name].append(verb.name) 
                                actors_before = []                           
                        names_str = print_names(actors_before)
                        for a in actors_before:
                            actors_actions[a.name].append("ORNOUN_" + names_str + str(order_cnt))
                            actors_actions[a.name].append(verb.name)
                        print(groups)
                        order_cnt+=1

 
                    #no choice
                    else:
                        for actor in task.nouns:                                               
                            names_str = print_names(task.nouns)
                            actors_actions[actor.name].append("BARRIER_" + names_str + str(order_cnt))
                            actors_actions[actor.name].append(verb.name)
                            actors_actions[actor.name].append("SINC_"+ names_str + str(order_cnt))
                        order_cnt+=1
        else:
            if (is_or_exist(task.verbs)):
                verbs_str = print_names(task.verbs)
                for actor in task.nouns:                             
                    if actor.name not in actors_actions:
                        actors_actions[actor.name] = []
                        actors_actions[actor.name].append("ORVERB_"+ verbs_str  + str(order_cnt))
            else:
                actor = task.nouns[0]
                for verb in task.verbs:
                    actors_actions[actor.name].append(verb.name)
    print(actors_actions)
    return actors_actions

input_text ="Robot A, Robot B and Robot C walk."
doc = nlp(input_text)
get_parallel_tasks(doc)



defaultdict(<class 'list'>, {'A': ['BARRIER_A_B_C_0', 'walk', 'SINC_A_B_C_0'], 'B': ['BARRIER_A_B_C_0', 'walk', 'SINC_A_B_C_0'], 'C': ['BARRIER_A_B_C_0', 'walk', 'SINC_A_B_C_0']})


defaultdict(list,
            {'A': ['BARRIER_A_B_C_0', 'walk', 'SINC_A_B_C_0'],
             'B': ['BARRIER_A_B_C_0', 'walk', 'SINC_A_B_C_0'],
             'C': ['BARRIER_A_B_C_0', 'walk', 'SINC_A_B_C_0']})

In [13]:
input_text = "Bob waters the plants, before Anna cleans the room. Anna jumps."
 
doc = nlp(input_text)
get_parallel_tasks(doc)

[<__main__.Task object at 0x3023a0410>, <__main__.Task object at 0x3023a0f50>]
defaultdict(<class 'list'>, {'Bob': ['waters', 'BARRIER_Anna_Bob_0'], 'Anna': ['BARRIER_Anna_Bob_0', 'cleans', 'jumps']})


defaultdict(list,
            {'Bob': ['waters', 'BARRIER_Anna_Bob_0'],
             'Anna': ['BARRIER_Anna_Bob_0', 'cleans', 'jumps']})

In [14]:
def transition_exists(net, name):
    try:
        net.transition(name)  # Try to retrieve the transition
        return True  
    except ConstraintError:
        return False


def place_exists(net, name):
    try:
        net.place(name)  # Try to retrieve the transition
        return True  
    except ConstraintError:
        return False

def extract_brenching(text):
    pattern = r"ORVERB_([a-zA-Z_]+(?:_[a-zA-Z_]+)*)_"
    words = re.findall(pattern, text)
    return words[0].split('_') if words else []

def extract_ornoun(text):
    pattern = r"^(ORNOUN)_([^_]+(?:_[^_]+)*)_([^_]+)$"  # Captures middle and last parts
    match = re.match(pattern, text)
    
    if match:
        final_part = f"{match.group(1)}_{match.group(3)}"  # ORNOUN_ + last word
        middle_part = match.group(2)  # The middle words
        return final_part, middle_part
    return None, None  # Return None if no match is found


def draw_petri_net(actors_actions):
    n = PetriNet('new')
    n.add_place(Place('start'))
    n.add_transition(Transition('task_start'))
    n.add_input('start', 'task_start', Value("ε"))
    
    n.add_place(Place('end'))
    n.add_transition(Transition('task_end'))
    n.add_output('end', "task_end", Value("ε"))


    for actor in actors_actions.keys():
        #start place and transition
        n.add_place(Place(actor+str(-1), "Robot is on"))
        previous_place = actor+str(-1)
        n.add_output(actor+str(-1), 'task_start', Value("ε"))
        
        i = 0
        while i < len(actors_actions[actor]):
            verb = actors_actions[actor][i]
            
            #for sequential
            if not verb.startswith(("BARRIER_", "SINC_", "OR")):
                n.add_transition(Transition(verb+'_'+actor+'_'+str(i)))
            
                n.add_input(previous_place, verb+'_'+actor+'_'+str(i), Value("ε"))
                n.add_place(Place(actor+str(i)))
                n.add_output(actor+str(i), verb+'_'+actor+'_'+str(i), Value("ε"))
                
                previous_place = actor+str(i)
                    
            else:
            #for barriers
                if verb.startswith(("BARRIER_")):
                    if not transition_exists(n, verb):
                        n.add_transition(Transition(verb))
                    n.add_input(previous_place, verb, Value("ε"))
                    n.add_place(Place(actor+str(i)))
                    n.add_output(actor+str(i), verb, Value("e"))
                    previous_place = actor+str(i)                    
        

                if verb.startswith(("SINC_")):    
                    verb = actors_actions[actor][i]
                    if not transition_exists(n, verb):
                        n.add_transition(Transition(verb))
                    n.add_input(previous_place, verb, Value("ε"))
                    n.add_place(Place(actor+str(i)))
                    n.add_output(actor+str(i), verb, Value("ε"))
                    previous_place = actor+str(i)           
                    
                    
                #for OR between verbs
                if verb.startswith(("ORVERB_")):
                    #ORVERB_take_wash_0
                    brenches = extract_brenching(verb)
                    print(brenches)

                    n.add_place(Place(actor+str(i)))
                    for b in brenches:
                        transition_name = b+'_'+actor+'_'+str(i)
                        n.add_transition(Transition(transition_name))
                        n.add_input(previous_place, transition_name, Value("ε"))
                        n.add_output(actor+str(i), transition_name, Value("h"))
                    
                    previous_place = actor+str(i)  
                
                #for OR between nouns
                if verb.startswith(("ORNOUN_")):    
                    print("hehehe")
                    transition_name, actors_name = extract_ornoun(verb)
                    print(transition_name)
                    if not transition_exists(n, transition_name):
                        n.add_transition(Transition(transition_name))
                        n.add_place(Place("PLACE_"+transition_name))
                        n.add_output("PLACE_"+transition_name, transition_name, Value("e"))
                    n.add_input(previous_place, transition_name, Value("ε"))
                    previous_place = "PLACE_"+transition_name
                    i+=1
                    verb = actors_actions[actor][i]
                    transition_name_2 = verb+'_'+actors_name+'_'+str(i)
                    if not transition_exists(n, transition_name_2):
                        n.add_transition(Transition(transition_name_2))
                        n.add_input(previous_place, transition_name_2, Value("h"))
                    
                        place_name="PLACE_"+transition_name+'2'
                        if not place_exists(n, place_name):
                            n.add_place(Place(place_name))
                            n.add_transition(Transition(transition_name+'2'))
                            n.add_input(place_name, transition_name+'2', Value("ε"))
                        n.add_output(place_name, transition_name_2, Value("e"))
                    n.add_place(Place(actor+str(i)))
                    n.add_output(actor+str(i), transition_name+'2', Value("e"))
                    previous_place = actor+str(i)
                    
                                       
                    
            if i == len(actors_actions[actor]) - 1:
                n.add_input(previous_place, 'task_end', Value("ε"))     
            
            i += 1           

    
    n.draw("new.png")


#input_text = "Mary, Ben or Mark and John walk."
#input_text = "Mary, Ben or Mark walk."
#input_text = "Robot A and Robot B, or Robot A and Robot C walk."
#input_text = "Robot A is running. Robot B is jumping. Robot C is singing."
#input_text = "Robot A and Robot B take the pan or washes it"
doc = nlp(input_text)
input = get_parallel_tasks(doc)  
draw_petri_net(input)


[<__main__.Task object at 0x30340e550>, <__main__.Task object at 0x30340e890>]
defaultdict(<class 'list'>, {'Bob': ['waters', 'BARRIER_Anna_Bob_0'], 'Anna': ['BARRIER_Anna_Bob_0', 'cleans', 'jumps']})


In [15]:
n = PetriNet('ggg')
n.add_place(Place('start', [0]))
n.add_transition(Transition('t', Expression('x<5')))
n.add_input('start', 't', Variable('x'))
n.add_output('start', 't', Expression('x+1'))
n.draw("value-1.png")

Graph(['node_0', 'node_1'], [])

In [16]:
# Test
#OR cases

#successful
#input_text = "Mary and Ben or Mark and John walk."
input_text = "Mary, Ben or Mark and John walk."
#input_text = "Mary, Ben or Mark walk."
#input_text = "Robot A and Robot B, or Robot A and Robot C walk."
#input_text = "Robot A is running. Robot B is jumping. Robot C is singing."
#input_text = "Robot A and Robot B take the pan or washes it"

 
doc = nlp(input_text)
get_parallel_tasks(doc)




defaultdict(<class 'list'>, {'Mary': ['ORNOUN_Mary_0', 'walk'], 'Ben': ['ORNOUN_Ben_0', 'walk'], 'Mark': ['ORNOUN_Mark_John_0', 'walk'], 'John': ['ORNOUN_Mark_John_0', 'walk']})


defaultdict(list,
            {'Mary': ['ORNOUN_Mary_0', 'walk'],
             'Ben': ['ORNOUN_Ben_0', 'walk'],
             'Mark': ['ORNOUN_Mark_John_0', 'walk'],
             'John': ['ORNOUN_Mark_John_0', 'walk']})

In [17]:
# Test

#input_text = "Robot A and Robot B take the pot. Robot C takes the spoon. Robots A and Robot C help each other"
#input_text = "Robot A takes the pan, Robot B takes the pencil."
#input_text = "Robot A takes the pan and drops it. "
#input_text = "Robot A, Robot B and Robot C takes the pan, Robot B takes the pencil"
#input_text = "Robot A and Robot B take the pot. Robot C takes the spoon. Robot A and Robot C work together and Robot B is cleaning"
#input_text = "Robot A is running, then swiming. Robot B is jumping. Robot C is singing."
#input_text = "Robot A, Robot B and Robot C take the pan and wash it"
#input_text = "Robot A and Robot B take the pan or wash it"


#WEIRD BEHAVIOUR in spacy
#input_text = "Robots A and Robot C help each other and Robot B is cleaning"
#input_text = "Robots A and Robot C help each other, Robot B cleans"

#input_text = "Robot A goes."
#input_text = "Robot G moves its arm while analyzing the object."
#input_text = "Robot A and Robot B take the pan or wash it"
#input_text = "Bob waters the plants, while Anna cleans the room."
#input_text = "Robot A, Robot B and Robot C take the pan and wash it"
input_text = "Robot A, Robot B and Robot C take the pan and wash it"


doc = nlp(input_text)
all_tasks = analyse_tree(doc)
for i in all_tasks:
    print(i)

print(get_parallel_tasks(doc))

#options={"compact": True, "distance":60}
#spacy.displacy.serve(doc, style="dep", auto_select_port=True)
#print_tasks(doc)


Task(Verbs: [take, Prepositions: (and, TAG: cc), TAG: ROOT, wash, TAG: conj], Nouns: [A, B, (and, TAG: cc), C])
defaultdict(<class 'list'>, {'A': ['BARRIER_A_B_C_0', 'take', 'SINC_A_B_C_0', 'BARRIER_A_B_C_1', 'wash', 'SINC_A_B_C_1'], 'B': ['BARRIER_A_B_C_0', 'take', 'SINC_A_B_C_0', 'BARRIER_A_B_C_1', 'wash', 'SINC_A_B_C_1'], 'C': ['BARRIER_A_B_C_0', 'take', 'SINC_A_B_C_0', 'BARRIER_A_B_C_1', 'wash', 'SINC_A_B_C_1']})
defaultdict(<class 'list'>, {'A': ['BARRIER_A_B_C_0', 'take', 'SINC_A_B_C_0', 'BARRIER_A_B_C_1', 'wash', 'SINC_A_B_C_1'], 'B': ['BARRIER_A_B_C_0', 'take', 'SINC_A_B_C_0', 'BARRIER_A_B_C_1', 'wash', 'SINC_A_B_C_1'], 'C': ['BARRIER_A_B_C_0', 'take', 'SINC_A_B_C_0', 'BARRIER_A_B_C_1', 'wash', 'SINC_A_B_C_1']})


In [18]:
#input_text = "Robot A and Robot B take the pot. Robot C takes the spoon. Robots A and Robot C help each other"
#input_text = "Robot A takes the pan, Robot B takes the pencil."
#input_text = "Robot A takes the pan and drops it. "
#input_text = "Robot A, Robot B and Robot C takes the pan, Robot B takes the pencil"
#input_text = "Robot A and Robot B take the pot. Robot C takes the spoon. Robot A and Robot C work together and Robot B is cleaning"
#input_text = "Robot A is running, then swiming. Robot B is jumping. Robot C is singing."
#input_text = "Robot A, Robot B and Robot C take the pan and wash it"
#input_text = "Robot A and Robot B take the pan or wash it"
#input_text = "Robot A takes the pan or washes it"
#input_text = "Robot A and Robot B take the pan or wash it"


input_text = "Bob waters the plants, before Anna cleans the room. Anna jumps."



doc = nlp(input_text)
input = get_parallel_tasks(doc)  
draw_petri_net(input)

[<__main__.Task object at 0x3022ada90>, <__main__.Task object at 0x3022ae190>]
defaultdict(<class 'list'>, {'Bob': ['waters', 'BARRIER_Bob_Anna_0'], 'Anna': ['BARRIER_Bob_Anna_0', 'cleans', 'jumps']})


In [19]:


class PetriNetApp:
    def __init__(self, root):
        self.root = root
        self.root.geometry("800x600")  # Set the window size to 800x600 (you can adjust the values as needed)

        self.entry = tk.Entry(root, width=50)
        self.entry.pack(pady=10)

        self.button = tk.Button(root, text="Draw Petri Net", command=self.on_button_click)
        self.button.pack(pady=5)

        self.canvas_frame = tk.Frame(root)
        self.canvas_frame.pack(fill="both", expand=True)

        self.canvas = tk.Canvas(self.canvas_frame)
        self.canvas.pack(side="left", expand=True, fill="both")

        self.scrollbar_y = tk.Scrollbar(self.canvas_frame, orient="vertical", command=self.canvas.yview)
        self.scrollbar_y.pack(side="right", fill="y")

        self.scrollbar_x = tk.Scrollbar(self.canvas_frame, orient="horizontal", command=self.canvas.xview)
        self.scrollbar_x.pack(side="bottom", fill="x")

        self.canvas.config(yscrollcommand=self.scrollbar_y.set, xscrollcommand=self.scrollbar_x.set)

        self.gif_frames = []  # Store frames of the gif
        self.current_frame = 0  # Index of the current frame
        self.gif_running = False  # To track if GIF animation should continue

    def on_button_click(self):
        if not self.entry.winfo_exists(): 
            return

        input_text = self.entry.get()
        self.display_loading_gif()
        threading.Thread(target=self.process_input, args=(input_text,)).start()

    def display_loading_gif(self):
        # Load the gif and split it into frames
        self.gif_frames = self.load_gif_frames("load.gif")
        self.current_frame = 0
        self.gif_running = True 
        self.animate_gif()

    def load_gif_frames(self, gif_path):
        img = Image.open(gif_path)
        frames = []
        try:
            while True:
                frame = img.copy()
                frames.append(ImageTk.PhotoImage(frame))
                img.seek(img.tell() + 1)
        except EOFError:
            pass 
        return frames

    def animate_gif(self):
        if self.gif_running and self.gif_frames:
            self.canvas.delete("all")
            self.canvas.create_image(0, 0, anchor="nw", image=self.gif_frames[self.current_frame])
            self.canvas.config(scrollregion=self.canvas.bbox("all"))

            self.current_frame = (self.current_frame + 1) % len(self.gif_frames)
            self.root.after(100, self.animate_gif)  # Update every 100ms

    def process_input(self, input_text):
        time.sleep(3)  # Simulate 3 seconds of processing time

        self.present_output(input_text)

        self.gif_running = False  # Stop the GIF animation
        self.root.after(0, self.display_image, "new.png")

    def present_output(self, input_text):
        doc = nlp(input_text)  
        input_data = get_parallel_tasks(doc)  
        draw_petri_net(input_data) 

    def display_image(self, image_path):
        if os.path.exists(image_path):
            img = Image.open(image_path)
            img_tk = ImageTk.PhotoImage(img)

            self.canvas.delete("all")
            self.canvas.create_image(0, 0, anchor="nw", image=img_tk)
            self.canvas.config(scrollregion=self.canvas.bbox("all"))
            self.canvas.image = img_tk 



In [24]:
root = tk.Tk()
app = PetriNetApp(root)
root.mainloop()